Based what items a user rated, a user-user similarity matrix can be build. Each user is represented as a vector, and cosine similarity is used to quantify how similiar two users are. 
Let's say if we want to recommend movies to Bob, we can then look up similiar users as Bob, pick up 10 most similar user using KNN and recommend items the other 10 similar users like but Bob hasn't seen yet.

In [1]:
from MovieLens import MovieLens
from surprise import KNNBasic
import heapq
from collections import defaultdict
from operator import itemgetter
from surprise.model_selection import LeaveOneOut
from RecommenderMetrics import RecommenderMetrics
from EvaluationData import EvaluationData

In [2]:
def LoadMovieLensData():
    ml=MovieLens()
    data=ml.loadMovieLensLatestSmall()
    print('computing movie popularity ranks, later we will use this to measure novelty...')
    rankings = ml.getPopularityRanks()
    return (ml, data, rankings)

In [3]:
ml, data, rankings = LoadMovieLensData()

computing movie popularity ranks, later we will use this to measure novelty...


In [4]:
evalData = EvaluationData(data, rankings)

In [5]:
trainSet= evalData.GetLOOCVTrainSet()

In [6]:
# calculate the similarity matrix. Using surprise package.

sim_options= {'name':'cosine','user_based': True}

model=KNNBasic(sim_options=sim_options)
model.fit(trainSet)
simMatrix= model.compute_similarities()

print(simMatrix[0][:10])


Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
[1.         0.98       0.88435552 0.95739835 0.88131273 0.963423
 0.94722266 0.93977491 0.94050468 0.95418766]


In [7]:
leftOutTestSet= evalData.GetLOOCVTestSet()

In [11]:
# get topN recommendations

topN= defaultdict(list)
k=10

for uiid in range(trainSet.n_users):
    # get top N similar users to this one
    similarityRow=simMatrix[uiid]
    
    similarUsers=[]
    for innerID, similarity_score in enumerate(similarityRow):
        if innerID != uiid:
            similarUsers.append((innerID, similarity_score))
    KNeighbors = heapq.nlargest(k, similarUsers, key=lambda x: x[1])
    
    # get the stuff they rated, and add up ratings for each item, weighted by user similarity
    
    candidates = defaultdict(float)
    for user in KNeighbors:
        SimilarityScore=user[1]
        for ratings in trainSet.ur[user[0]]:
            candidates[ratings[0]] += ratings[1]/5.0*SimilarityScore
            
    # build a dictionary of movies the subject has already watched
    watched={}
    for item, rating in trainSet.ur[uiid]:
        watched[item]=1
        
    # Get top-rated items from similar users that our subject hasn't watched yet: top 40 recommendations

    loop=0
    for item, ratingSum in sorted(candidates.items(),key=itemgetter(1),reverse=True):
        if item not in watched:
            movie_id=trainSet.to_raw_iid(item)
            topN[int(trainSet.to_raw_uid(uiid))].append((int(movie_id), ratingSum))
            loop+=1
            if loop > 40:
                break
    

# evaluating topN using hit rate:


In [12]:
print("HR",RecommenderMetrics.HitRate(topN, leftOutTestSet))

HR 0.04918032786885246
